In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import glob
import os

import itertools
from astropy.stats import sigma_clip

from tqdm import tqdm
import time

In [2]:
def ADC_convert(signal, gain, offset):
    signal = signal.astype(np.float64)
    signal /= gain
    signal += offset
    return signal

def mask_hot_dead(signal, dead, dark):
    hot = sigma_clip(
        dark, sigma=5, maxiters=5
    ).mask
    hot = np.tile(hot, (signal.shape[0], 1, 1))
    dead = np.tile(dead, (signal.shape[0], 1, 1))
    signal = np.ma.masked_where(dead, signal)
    signal = np.ma.masked_where(hot, signal)
    return signal


def apply_linear_corr(linear_corr,clean_signal):
    linear_corr = np.flip(linear_corr, axis=0)
    for x, y in itertools.product(
                range(clean_signal.shape[1]), range(clean_signal.shape[2])
            ):
        poli = np.poly1d(linear_corr[:, x, y])
        clean_signal[:, x, y] = poli(clean_signal[:, x, y])
    return clean_signal

def clean_dark(signal, dead, dark, dt):

    dark = np.ma.masked_where(dead, dark)
    dark = np.tile(dark, (signal.shape[0], 1, 1))

    signal -= dark* dt[:, np.newaxis, np.newaxis]
    return signal

def get_cds(signal):
    cds = signal[:,1::2,:,:] - signal[:,::2,:,:]
    return cds

def bin_obs(cds_signal,binning):
    cds_transposed = cds_signal.transpose(0,1,3,2)
    cds_binned = np.zeros((cds_transposed.shape[0], cds_transposed.shape[1]//binning, cds_transposed.shape[2], cds_transposed.shape[3]))
    for i in range(cds_transposed.shape[1]//binning):
        cds_binned[:,i,:,:] = np.sum(cds_transposed[:,i*binning:(i+1)*binning,:,:], axis=1)
    return cds_binned

def correct_flat_field(flat,dead, signal):
    flat = flat.transpose(1, 0)
    dead = dead.transpose(1, 0)
    flat = np.ma.masked_where(dead, flat)
    flat = np.tile(flat, (signal.shape[0], 1, 1))
    signal = signal / flat
    return signal

def get_cds_origial(signal):
    """
    calcs difference in time becuase the detector is read twice, once in the beginning and once in the end of the measurement, the difference is what's measured
    """
    cds = signal[1::2,:,:] - signal[::2,:,:]
    return cds

In [11]:
planet_id = 612015401

path_folder = ""

train_adc_info = pd.read_csv(os.path.join(path_folder, 'train_adc_info.csv'))
train_adc_info = train_adc_info.set_index('planet_id')
axis_info = pd.read_parquet(os.path.join(path_folder,'axis_info.parquet'))

DO_MASK = True  # filter out non responsive pixels
DO_THE_NL_CORR = True # most time consuming step, you can choose to ignore it for rapid prototyping, nonlinear correction due to artefacts when reading pixels
DO_DARK = True  # dark current is accumulating over time in the pixels, need to compensate that (seems like integration artefact)
DO_FLAT = True  # pixel to pixel variation correction (e.g. how pixels respond differently when illuminated uniformly)
TIME_BINNING = False  #do a time binning on choosen frequency

cut_inf, cut_sup = 39, 321
l = cut_sup - cut_inf

t0 = time.time()
cut_inf, cut_sup = 39, 321
l = cut_sup - cut_inf
AIRS_CH0_clean = np.zeros((1, 11250, 32, l))
FGS1_clean = np.zeros((1, 135000, 12, 12))
df = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_signal.parquet'))
signal = df.values.astype(np.float64).reshape((df.shape[0], 32, 356))
gain = train_adc_info['AIRS-CH0_adc_gain'].loc[planet_id]
offset = train_adc_info['AIRS-CH0_adc_offset'].loc[planet_id]
signal = ADC_convert(signal, gain, offset)
dt_airs = axis_info['AIRS-CH0-integration_time'].dropna().values
dt_airs[1::2] += 0.1
chopped_signal = signal[:, :, cut_inf:cut_sup]
airs_original = chopped_signal
del signal, df
# CLEANING THE DATA: AIRS
flat = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_calibration/flat.parquet')).values.astype(np.float64).reshape((32, 356))[:, cut_inf:cut_sup]
dark = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_calibration/dark.parquet')).values.astype(np.float64).reshape((32, 356))[:, cut_inf:cut_sup]
dead_airs = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_calibration/dead.parquet')).values.astype(np.float64).reshape((32, 356))[:, cut_inf:cut_sup]
linear_corr = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_calibration/linear_corr.parquet')).values.astype(np.float64).reshape((6, 32, 356))[:, :, cut_inf:cut_sup]
t1 = time.time()
if DO_MASK:
    chopped_signal = mask_hot_dead(chopped_signal, dead_airs, dark)
    AIRS_CH0_clean[0] = chopped_signal
else:
    AIRS_CH0_clean[0] = chopped_signal
t2 = time.time()
if DO_THE_NL_CORR: 
    linear_corr_signal = apply_linear_corr(linear_corr,AIRS_CH0_clean[0])
    AIRS_CH0_clean[0] = linear_corr_signal
del linear_corr
t3 = time.time()
if DO_DARK: 
    cleaned_signal = clean_dark(AIRS_CH0_clean[0], dead_airs, dark,dt_airs)
    AIRS_CH0_clean[0] = cleaned_signal
else: 
    pass
del dark
t4 = time.time()
df = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_signal.parquet'))
fgs_signal = df.values.astype(np.float64).reshape((df.shape[0], 32, 32))
print(fgs_signal.shape)
fgs_signal = fgs_signal[:,10:22,10:22]
print(fgs_signal.shape)
FGS1_gain = train_adc_info['FGS1_adc_gain'].loc[planet_id]
FGS1_offset = train_adc_info['FGS1_adc_offset'].loc[planet_id]
fgs_signal = ADC_convert(fgs_signal, FGS1_gain, FGS1_offset)
dt_fgs1 = np.ones(len(fgs_signal))*0.1  ## please refer to data documentation for more information
dt_fgs1[1::2] += 0.1
chopped_FGS1 = fgs_signal
fgs_original = chopped_FGS1
del fgs_signal, df
# CLEANING THE DATA: FGS1
dark = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_calibration/dark.parquet')).values.astype(np.float64).reshape((32, 32))
dark = dark[10:22,10:22]
dead_fgs1 = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_calibration/dead.parquet')).values.astype(np.float64).reshape((32, 32))
dead_fgs1 = dead_fgs1[10:22,10:22]
linear_corr = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_calibration/linear_corr.parquet')).values.astype(np.float64).reshape((6, 32, 32))
linear_corr = linear_corr[:,10:22,10:22]
t5 = time.time()
if DO_MASK:
    chopped_FGS1 = mask_hot_dead(chopped_FGS1, dead_fgs1, dark)
    FGS1_clean[0] = chopped_FGS1
else:
    FGS1_clean[0] = chopped_FGS1
t6 = time.time()
if DO_THE_NL_CORR: 
    linear_corr_signal = apply_linear_corr(linear_corr,FGS1_clean[0])
    FGS1_clean[0,:, :, :] = linear_corr_signal
del linear_corr
t7 = time.time()
if DO_DARK: 
    cleaned_signal = clean_dark(FGS1_clean[0], dead_fgs1, dark,dt_fgs1)
    FGS1_clean[0] = cleaned_signal
else: 
    pass
del dark 
t8 = time.time()
AIRS_cds = get_cds(AIRS_CH0_clean)
FGS1_cds = get_cds(FGS1_clean)
t9 = time.time()
## (Optional) calc diff of data between two timestamps
if TIME_BINNING:
    AIRS_cds_binned = bin_obs(AIRS_cds,binning=30)
    FGS1_cds_binned = bin_obs(FGS1_cds,binning=30*12)
else:
    AIRS_cds = AIRS_cds.transpose(0,1,3,2) ## this is important to make it consistent for flat fielding, but you can always change it
    AIRS_cds_binned = AIRS_cds
    FGS1_cds = FGS1_cds.transpose(0,1,3,2)
    FGS1_cds_binned = FGS1_cds
del AIRS_cds, FGS1_cds
flat_airs = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/AIRS-CH0_calibration/flat.parquet')).values.astype(np.float64).reshape((32, 356))[:, cut_inf:cut_sup]
flat_fgs = pd.read_parquet(os.path.join(path_folder,f'train/{planet_id}/FGS1_calibration/flat.parquet')).values.astype(np.float64).reshape((32, 32))
flat_fgs = flat_fgs[10:22,10:22]
if DO_FLAT:
    corrected_AIRS_cds_binned = correct_flat_field(flat_airs,dead_airs, AIRS_cds_binned[0])
    AIRS_cds_binned[0] = corrected_AIRS_cds_binned
    corrected_FGS1_cds_binned = correct_flat_field(flat_fgs,dead_fgs1, FGS1_cds_binned[0])
    FGS1_cds_binned[0] = corrected_FGS1_cds_binned
else:
    pass
AIRS_cds_original = np.expand_dims(get_cds_origial(airs_original), axis=0).transpose(0,1,3,2)
FGS1_cds_original = np.expand_dims(get_cds_origial(fgs_original), axis=0).transpose(0,1,3,2)
t10 = time.time()
deltaT = t10-t0
print('t1',(t1-t0)/deltaT)
print('t2',(t2-t1)/deltaT)
print('t3',(t3-t2)/deltaT)
print('t4',(t4-t3)/deltaT)
print('t5',(t5-t4)/deltaT)
print('t6',(t6-t5)/deltaT)
print('t7',(t7-t6)/deltaT)
print('t8',(t8-t7)/deltaT)
print('t9',(t9-t8)/deltaT)
print('t10',(t10-t9)/deltaT)
print(deltaT)

(135000, 32, 32)
(135000, 12, 12)
t1 0.1487279585246287
t2 0.13750977556198257
t3 0.42932391667551956
t4 0.04308983421711504
t5 0.04686971473161475
t6 0.01077246891056976
t7 0.07678232855527897
t8 0.005850517337125006
t9 0.008335260811693362
t10 0.09273822467447226
11.510809183120728


In [ ]:
AIRS_cds_binned.shape, FGS1_cds_binned.shape

In [ ]:
np.sum(AIRS_cds_binned, axis = 3).shape

In [ ]:
# 800 mb for just airs full data vs 120mb parquet file vs 25mb when summed along wavelength
# 1gb for fsg data vs 120mb parquet vs. 33mb summed along wavelength

np.savez('airs.npz', a=np.sum(FGS1_cds_binned, axis = 3))